In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('data/processed_cars_data.csv')
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4979 entries, 0 to 4978
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   car name         4979 non-null   object 
 1   engine_capacity  4979 non-null   float64
 2   cylinder         4979 non-null   int64  
 3   horse_power      4979 non-null   int64  
 4   top_speed        4979 non-null   int64  
 5   brand            4979 non-null   object 
 6   country          4979 non-null   object 
 7   num_seats        4979 non-null   int64  
 8   Price_USD        4979 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 350.2+ KB


,engine_capacity,cylinder,horse_power,top_speed,num_seats,Price_USD
count,4979.000000,4979.000000,4979.000000,4979.000000,4979.000000,4.979000e+03
mean,3.616951,5.165696,285.849367,222.112874,4.921068,7.215546e+04
std,28.331114,1.842988,183.686725,46.665063,1.401370,1.072890e+05
min,0.000000,3.000000,25.000000,120.000000,2.000000,4.608000e+03
25%,2.000000,4.000000,164.000000,188.000000,5.000000,2.565000e+04
50%,2.400000,4.000000,250.000000,220.000000,5.000000,4.725000e+04
75%,3.500000,6.000000,362.000000,250.000000,5.000000,8.343765e+04
max,1000.000000,16.000000,5050.000000,966.000000,18.000000,3.594996e+06


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Selecting features and target variable
X = df[['engine_capacity', 'cylinder', 'horse_power', 'top_speed', 'num_seats']]
y = df['Price_USD']


# Splitting the dataset into training (60%), validation (20%), and test (20%) sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Initialize the StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Adding a bias term to each split
X_train_bias = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]
X_val_bias = np.c_[np.ones(X_val_scaled.shape[0]), X_val_scaled]
X_test_bias = np.c_[np.ones(X_test_scaled.shape[0]), X_test_scaled]

# Displaying the shapes of the splits to confirm
X_train_bias.shape, X_val_bias.shape, X_test_bias.shape, y_train.shape, y_val.shape, y_test.shape


((2987, 6), (996, 6), (996, 6), (2987,), (996,), (996,))